Self-Attention的公式
$$\text{SelfAttention}(X) = \text{softmax}\left(\frac{Q \cdot K}{\sqrt{d}}\right) \cdot V$$
$Q = K = V = W \times X$  ，其中Q K V 对应不同的矩阵 W


### 补充知识点
1. matmul 和 @ 符号是一样的作用
2. 为什么要除以$\sqrt{d}$    a. 防止梯度消失 b. 为了让 QK 的内积分布保持和输入一样
3. 爱因斯坦方程表达式用法：torch.einsum("bqd,bkd-> bqk", X, X).shape
4. X.repeat(1, 1, 3) 表示在不同的维度进行 repeat操作，也可以用 tensor.expand 操作

### 第一层：简化版
- 直接对着公式实现  $\text{SelfAttention}(X) = \text{softmax}\left(\frac{Q \cdot K}{\sqrt{d}}\right) \cdot V$

In [11]:
import math
import torch
import torch.nn as nn

class SelfAttenionV1(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim=hidden_dim
        # 一般 Linear 都是默认有 bias
        # 一般来说， input dim 的 hidden dim
        self.query_proj=nn.Linear(hidden_dim,hidden_dim)
        self.key_proj=nn.Linear(hidden_dim,hidden_dim)
        self.valye_proj=nn.Linear(hidden_dim,hidden_dim)
    
    def forward(self,X):
        # X shape is: (batch, seq_len, hidden_dim)， 一般是和 hidden_dim 相同
        # 但是 X 的 final dim 可以和 hidden_dim 不同
        Q=self.query_proj(X)
        K=self.key_proj(X)
        V=self.valye_proj(X)
        # shape is: (batch, seq_len, seq_len)
        # torch.matmul 可以改成 Q @ K.T
        # 其中 K 需要改成 shape 为： (batch, hidden_dim, seq_len)
        attention_value=torch.matmul(Q,K.transpose(-1,-2))
        attention_weight=torch.softmax(attention_value/math.sqrt(self.hidden_dim),dim=-1)

        print(attention_weight)
        # shape is: (batch, seq_len, hidden_dim)
        output=torch.matmul(attention_weight,V)
        return output

X=torch.rand(3,2,4)

net=SelfAttenionV1(X.shape[-1])
net(X)

tensor([[[0.4714, 0.5286],
         [0.4424, 0.5576]],

        [[0.4997, 0.5003],
         [0.4813, 0.5187]],

        [[0.4918, 0.5082],
         [0.4973, 0.5027]]], grad_fn=<SoftmaxBackward0>)


tensor([[[-0.0865, -0.5982,  0.1739,  0.2889],
         [-0.0796, -0.5996,  0.1771,  0.2939]],

        [[ 0.1381, -0.2920,  0.0974,  0.2454],
         [ 0.1341, -0.2924,  0.0998,  0.2431]],

        [[ 0.0957, -0.4515,  0.0607,  0.3143],
         [ 0.0945, -0.4525,  0.0609,  0.3142]]], grad_fn=<UnsafeViewBackward0>)

### 第二层: 效率优化
- 上面那哪些操作可以合并矩阵优化呢？- QKV 矩阵计算的时候，可以合并成一个大矩阵计算  
但是当前 transformers 实现中，其实是三个不同的 Linear 层

In [23]:
class SelfAttenionV2(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim=hidden_dim
        # 在维度较小情况下可以合并计算，但是Llama, qwen, gpt的参数很大，还是分开计算
        self.proj=nn.Linear(hidden_dim,hidden_dim*3)
    def forward(self,X):
        # X shape is: (batch, seq, dim)
        QKV=self.proj(X) # (batch, seq, dim * 3)
        # reshape 从希望的 q, k, v的形式
        Q,K,V=torch.split(QKV,self.hidden_dim,dim=-1)
        print(Q.shape)
        print(K.transpose(-1,-2).shape)
        att_weight=torch.softmax((Q @ K.transpose(-1,-2))/math.sqrt(self.hidden_dim),dim=-1)
        output=att_weight @ V
        return output
         
X=torch.rand(3,2,4)
net=SelfAttenionV2(X.shape[-1])
net(X)



torch.Size([3, 2, 4])
torch.Size([3, 4, 2])


tensor([[[-0.5421, -1.0168, -0.3451, -0.4420],
         [-0.5412, -1.0166, -0.3441, -0.4415]],

        [[-0.5084, -0.9868, -0.3949, -0.4141],
         [-0.5059, -0.9880, -0.3945, -0.4135]],

        [[-0.4093, -0.9306, -0.1960, -0.3420],
         [-0.4085, -0.9300, -0.1953, -0.3413]]], grad_fn=<UnsafeViewBackward0>)

### 第三重: 加入细节
- 看上去 self attention 实现很简单，但里面还有一些细节，还有哪些细节呢？
- attention 计算的时候有 dropout，而且是比较奇怪的位置
- attention 计算的时候一般会加入 attention_mask，因为样本会进行一些 padding 操作；
- MultiHeadAttention 过程中，除了 QKV 三个矩阵之外，还有一个 output 对应的投影矩阵

In [74]:
import torch
import torch.nn as nn
import math

class SelfAttentionV3(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.proj = nn.Linear(hidden_dim, hidden_dim * 3)
        self.att_drop = nn.Dropout(0.1)

    def forward(self, X, attention_mask=None):
        # X shape: (batch, seq, dim)
        QKV = self.proj(X)  # (batch, seq, dim * 3)
        Q, K, V = torch.split(QKV, self.hidden_dim, dim=-1)  # 每个形状: (batch, seq, dim)

        # 计算注意力权重
        att_weight = Q @ K.transpose(-1, -2) / math.sqrt(self.hidden_dim)  # (batch, seq, seq)

        if attention_mask is not None:
            # 确保 attention_mask 是布尔张量
            attention_mask = attention_mask == 0  # 将 0 转换为 True，其他值转换为 False
            # 使用 masked_fill 填充极小值
            print("att_weight:",att_weight.shape)
            print("attention_mask:",attention_mask.shape)

            att_weight = att_weight.masked_fill(attention_mask, float("-1e20"))

        # 计算 softmax
        att_weight = torch.softmax(att_weight, dim=-1)  # (batch, seq, seq)
        att_weight = self.att_drop(att_weight)

        # 计算输出
        output = att_weight @ V  # (batch, seq, dim)
        return output

# 测试数据
X = torch.rand(3, 4, 2)  # (batch_size=3, seq_len=4, hidden_dim=2)
b = torch.tensor(
    [
        [1, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 0, 0],
    ]
)  # (batch_size=3, seq_len=4)
mask = b.unsqueeze(dim=1).repeat(1, 4, 1)  # (batch_size=3, seq_len=4, seq_len=4)

# 初始化网络
net = SelfAttentionV3(X.shape[-1])
output = net(X, mask)
print(output.shape)  # 应该输出: torch.Size([3, 4, 2])


att_weight: torch.Size([3, 4, 4])
attention_mask: torch.Size([3, 4, 4])
torch.Size([3, 4, 2])


### 第四层：完整写法

In [ ]:
import torch
import torch.nn as nn
import math

# 定义常量
DROPOUT_PROB = 0.1

class SelfAttentionV4(nn.Module):
    def __init__(self, dim) -> None:
        super().__init__()
        self.dim = dim

        # 定义线性变换层
        self.query_proj = nn.Linear(dim, dim)
        self.key_proj = nn.Linear(dim, dim)
        self.value_proj = nn.Linear(dim, dim)
        
        # 定义 dropout 层
        self.attention_dropout = nn.Dropout(DROPOUT_PROB)


    def forward(self, X, attention_mask=None):
        """
        X: 输入张量，形状为 (batch_size, seq_len, dim)
        attention_mask: 注意力掩码，形状为 (batch_size, seq_len)
        """
        # 计算 Q, K, V
        Q = self.query_proj(X)
        K = self.key_proj(X)
        V = self.value_proj(X)

        # 计算注意力权重
        att_weight = Q @ K.transpose(-1, -2) / math.sqrt(self.dim)
        
        # 应用注意力掩码
        if attention_mask is not None:
            # 给 masked 位置填充一个极小的值-1e20，然后取exp指数函数负无穷就变为0
            att_weight = att_weight.masked_fill(attention_mask == 0, float("-1e20"))

        # 计算 softmax
        att_weight = torch.softmax(att_weight, dim=-1)

        # 应用 dropout
        att_weight = self.attention_dropout(att_weight)

        # 计算加权和
        output = att_weight @ V
        return output

X = torch.rand(3, 4, 2)
b = torch.tensor(
    [
        [1, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 0, 0],
    ]
)
mask = b.unsqueeze(dim=1).repeat(1, 4, 1)
print("mask:",mask.shape)
net = SelfAttentionV4(X.shape[-1])
output=net(X, mask)
print("output:",output.shape)


mask: torch.Size([3, 4, 4])
output: torch.Size([3, 4, 2])
